In [1]:
from hmoe_buildkit import HMOEBuilder
model_names = [
    "HuggingFaceTB/SmolVLM-500M-Instruct",
    "meta-llama/Llama-3.2-1B",
    "Qwen/Qwen3-0.6B"
]
builder = HMOEBuilder(model_names)

Loading Tokenizers
Detected HuggingFaceTB/SmolVLM-500M-Instruct is a VLM.
Loading Models


In [2]:
hmoe_tokenizer, hmoe_model = builder.build()

Merging Tokenizers
Preparing HMOE
Mixing Embeddings


100%|██████████| 180211/180211 [00:12<00:00, 14214.69it/s]


Mixing LM Head


100%|██████████| 180211/180211 [00:12<00:00, 14280.18it/s]


In [3]:
from transformers.image_utils import load_image
image = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe this image?"}
        ]
    },
]

# Prepare inputs
prompt = hmoe_tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = hmoe_tokenizer(text=prompt, images=[image], return_tensors="pt")

In [4]:
print(hmoe_model)

HMOE(
  (embed_tokens): Embedding(180211, 960)
  (vision_encoder): Idefics3Model(
    (vision_model): Idefics3VisionTransformer(
      (embeddings): Idefics3VisionEmbeddings(
        (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
        (position_embedding): Embedding(1024, 768)
      )
      (encoder): Idefics3Encoder(
        (layers): ModuleList(
          (0-11): 12 x Idefics3EncoderLayer(
            (self_attn): Idefics3VisionAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics3VisionMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1)

In [5]:
import torch
inputs = inputs.to('cuda')
hmoe_model = hmoe_model.to('cuda')
with torch.no_grad():
    hmoe_model(**inputs)